In [1]:
import os, glob, csv, statistics, math, numpy as np, exifread
from osgeo import gdal, osr
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
from PIL import Image, ExifTags
import cameratransform as ct

base_dir = r"C:\Users\gdlarsen\Documents\UAS_survey-JHI_WE_S2"

In [2]:
# create a class that stores important photogrammetry parameters
# the only ones that aren't available in Sony metadata are sensor width
class Camera:
    def __init__(self, name, sensorW, focalL, imageW, imageH):
        # name, focal length in mm
        self.name, self.fl = name, focalL
        # sensor width, height, diagonal in mm
        self.sw, self.sh = sensorW, (imageH/imageW)*sensorW
        self.sd = math.hypot(sensorW, self.sh)
        # image width, height, diagonal in pixels
        self.imw, self.imh, self.imd = imageW, imageH, math.hypot(imageW, imageH)
        # angle of view width, heigh, diagonal in degrees
        self.aovw, self.aovh, self.aovd = math.degrees(2*math.atan(sensorW/(2*focalL))), math.degrees(2*math.atan(self.sh/(2*focalL))), math.degrees(2*math.atan(self.sd/(2*focalL)))

Sony_a6100 = Camera('Sony a6100', 23.5, 20, 6000, 4000)

In [3]:
# this function scrapes chunks of metadata for the Sony a6100
# it's the most I could find using native python methods
# (i.e. not calling ExifTool or anything else in teh background)

def scrape_metadata(img, lighting_metadata=True, gps_metadata=True, imu_metadata=True):
    metadata_package = {}
    with Image.open(img) as im:
        exif = im.getexif()
        exif_data = exif.get_ifd(0x8769)
        if lighting_metadata==True:
            lighting_metadata={}
            for tag_id in exif_data:
                tag = ExifTags.TAGS.get(tag_id, tag_id)
                content = [exif_data.get(tag_id)]
                if type(content).__name__ == 'IFDRational':
                    content = float(content)
                lighting_metadata[tag] = content
            del lighting_metadata['MakerNote'], lighting_metadata['FlashPixVersion'], lighting_metadata['FileSource'], lighting_metadata['SceneType'], lighting_metadata['ComponentsConfiguration']
            metadata_package.update(lighting_metadata)
        if gps_metadata==True:
            gps_metadata = {}
            for tag_id in ExifTags.IFD:
                ifd = exif.get_ifd(tag_id)
                if tag_id == ExifTags.IFD.GPSInfo:
                    resolve = ExifTags.GPSTAGS
                    for k, v in ifd.items():
                        tag = resolve.get(k, k)
                        gps_metadata[tag] = [v]
            del gps_metadata['GPSVersionID'], gps_metadata['GPSAltitudeRef']
            gps_metadata['GPSLongitudeDD'] = [(float(gps_metadata['GPSLongitude'][0][0]) + float(gps_metadata['GPSLongitude'][0][1])/60 + float(gps_metadata['GPSLongitude'][0][2])/(60*60))*(-1 if gps_metadata['GPSLongitudeRef'][0] == 'W' else 1)]
            gps_metadata['GPSLatitudeDD'] = [(float(gps_metadata['GPSLatitude'][0][0]) + float(gps_metadata['GPSLatitude'][0][1])/60 + float(gps_metadata['GPSLatitude'][0][2])/(60*60))*(-1 if gps_metadata['GPSLatitudeRef'][0] == 'S' else 1)]
            metadata_package.update(gps_metadata)
        if imu_metadata==True:
            xmpdata = im.getxmp()['xmpmeta']['RDF']['Description']
            imu_metadata = xmpdata
            metadata_package.update(imu_metadata)
    return metadata_package

In [4]:
# This sections reads the jpgs and extracts the metadata into a table
img_list = glob.glob(f"{base_dir}//*OUTPUT//*.jpg", recursive=True)

# img_list = img_list[355:365] # for demo

df = pd.DataFrame()
for img in img_list:
    image_name = img.split("\\")[-1].split(".")[0]
    metadata = scrape_metadata(img)
    sensor_size = Sony_a6100.sw, Sony_a6100.sh
    metadict = {'ImageName': [image_name], 'ImagePath': [img]}|metadata|{'SensorSize': [sensor_size]}
    df = pd.concat([df, pd.DataFrame.from_dict(metadict)])

In [5]:
# Altitude is notoriously inaccurate, whether from GPS or drone.
# In this case, from inspecting photos, it's clear that the drone
# was closer to the water than GPS estimated
# so we're using an offset to approximate the drone altitude instead

# read the altitude from the metadata table
altitude_from_photos = statistics.median([float(i) for i in df['GPSAltitude']])

# read the altitude from the CSV flight log
csv_list = glob.glob(f"{base_dir}//FLIGHT RECORD//*.csv", recursive=True)
print(f"using log file {csv_list[0]}")
csv_df = pd.read_csv(csv_list[0])
altitude_from_drone = statistics.median(csv_df['alt'])

# calculate the altitude offset
altitude_offset = altitude_from_drone-altitude_from_photos
print(f"using altitude offset of {altitude_offset} meters")

using log file C:\Users\gdlarsen\Documents\UAS_survey-JHI_WE_S2//FLIGHT RECORD\JHI_120624_ext_S2_F1 Flight 01.csv
using altitude offset of -11.073250000000002 meters


In [6]:
# This sections reads uses the metadata to generate GCPs for image projection

# Median GPSAltitude from metadata - median altitutde from flight log
# pulling metadata should be easy enough, maybe make a first step that assembles the table
# to be used with the footprints output
# but extracting from the ulog is a more complicated issue

df2 = pd.DataFrame()

for index, metadata in df.iterrows():
    # queue up important parameters from the metadata
    f = metadata['FocalLength']
    sensor_size = metadata['SensorSize']
    image_size = metadata['ExifImageWidth'], metadata['ExifImageHeight']
    lat, lon = metadata['GPSLatitudeDD'], metadata['GPSLongitudeDD']
    alt = float(metadata['GPSAltitude'])+altitude_offset
    yaw, pitch, roll = float(metadata['Yaw']), float(metadata['Pitch']), float(metadata['Roll'])

    # use cameratransform package to project image based on parameters
    cam = ct.Camera(ct.RectilinearProjection(focallength_mm = f, sensor = sensor_size, image = image_size),
                    ct.SpatialOrientation(elevation_m = alt, tilt_deg = pitch, roll_deg = roll, heading_deg = yaw, 
                                        pos_x_m = 0, pos_y_m = 0))
    
    # use cameratransform package to assign spatial values to image locations
    cam.setGPSpos(lat, lon, alt)
    coords = [cam.gpsFromImage([0, 0])[0:2], cam.gpsFromImage([image_size[0] - 1, 0])[0:2], 
                                cam.gpsFromImage([image_size[0] - 1, image_size[1] - 1])[0:2], cam.gpsFromImage([0, image_size[1] - 1])[0:2]]
    coords = [i.tolist() for i in coords] # necessary data conversion for later use in gdal's GCP tool
    img_w, img_h = image_size[0], image_size[1]
    # 1=top left, 2=top right, 3=bottom right, 4=bottom left
    # 4-8 are midpoints of quadralateral, to enable 2nd order transformation
    GCP_list = [[coords[0][1], coords[0][0], 0, 0, 0],
                [coords[1][1], coords[1][0], 0, img_w-1, 0],
                [coords[2][1], coords[2][0], 0, img_w-1, img_h-1],
                [coords[3][1], coords[3][0], 0, 0, img_h-1],
                [(coords[0][1]+coords[1][1])/2, (coords[0][0]+coords[1][0])/2, 0, (img_w-1)/2, 0],
                [(coords[1][1]+coords[2][1])/2, (coords[1][0]+coords[2][0])/2, 0, img_w-1, (img_h-1)/2],
                [(coords[2][1]+coords[3][1])/2, (coords[2][0]+coords[3][0])/2, 0, (img_w-1)/2, img_h-1],
                [(coords[3][1]+coords[0][1])/2, (coords[3][0]+coords[0][0])/2, 0, 0, (img_h-1)/2]]
    # bundle everything into a dictionary then append to dataframe
    metadict = {'ImageName': metadata['ImageName'], 'GCPList': [GCP_list], 'geometry': [Polygon([i[::-1] for i in coords])]}
    df2 = pd.concat([df2, pd.DataFrame.from_dict(metadict)])

gdf = gpd.GeoDataFrame(pd.merge(df, df2, on='ImageName'), crs="EPSG:4326")

In [7]:
# This section outputs the geodataframe to a shapefile.
# Note that metadata field names get truncated to 10 characters

# This section suppresses those warnings
import warnings
warnings.filterwarnings("ignore", message="Normalized/laundered field name")

out_dir = f"{base_dir}\\Shapefiles\\"
os.makedirs(out_dir, exist_ok = True)
out_path = f"{out_dir}Image_footprints.shp"
gdf.to_file(out_path)

C:\Users\gdlarsen\AppData\Local\Temp\1\ipykernel_44060\1330382994.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(out_path)
C:\Users\gdlarsen\AppData\Local\miniforge3\envs\spatial\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value '[[-137.06432590581912, 58.86442551137297, 0, 0, 0], [-137.06541079307658, 58.864872730278364, 0, 5999, 0], [-137.06483195029875, 58.865243035581145, 0, 5999, 3999], [-137.06375668445438, 58.864799812061726, 0, 0, 3999], [-137.06486834944786, 58.86464912082567, 0, 2999.5, 0], [-137.06512137168767, 58.865057882929754, 0, 5999, 1999.5], [-137.06429431737655, 58.86502142382143, 0, 2999.5, 3999], [-137.06404129513675, 58.864612661717345, 0, 0, 1999.5]]' of field GCPList has been truncated to 254 characters.  This warning will not be emitted any more for that layer.
  ogr_write(


In [ ]:
img_dir = f"{base_dir}\\OUTPUT\\"
out_dir = f"{base_dir}\\GeoreferencedTiffs\\"
os.makedirs(out_dir, exist_ok = True)

gdal.UseExceptions()

for index, record in gdf.iterrows():
    
    # Open the output file for writing for writing:
    ds = gdal.Open(record['ImagePath'])
    if ds is None:
        print(f"Could not open image: {record['ImagePath']}")
    # Set spatial reference:
    sr = osr.SpatialReference()
    sr.ImportFromEPSG(4326)

    # import GCP corners - there's got to be a better way to unpack i, but lists get rejected.
    gcps = [gdal.GCP(i[0], i[1], i[2], i[3], i[4]) for i in record['GCPList']]

    # Apply the GCPs to the open output file then warp it
    ds.SetGCPs(gcps, sr.ExportToWkt())

    kwargs = {'format': 'GTiff', 'polynomialOrder':2, 'srcNodata': '0,0,0', 'dstNodata': 'nodata'}
    ds = gdal.Warp(f"{out_dir}{record['ImageName']}_GeoRef.tif", ds, **kwargs)
    # Clear the variable (not sure if necessary, but good form)
    ds = None
    # counter, just because it can be a long process
    if (index+1) % 10 == 0:
        print(f"Completed file {index+1} out of {len(df)+1}")
print("finished processing images")

Completed file 10 out of 1685
Completed file 20 out of 1685
Completed file 30 out of 1685
Completed file 40 out of 1685
Completed file 50 out of 1685
Completed file 60 out of 1685
Completed file 70 out of 1685
